# Series temporales con Pandas

Uno de los usos más relevantes de Pandas en entorno financiero son las series de datos temporales. Trabajar con fechas y con series en el tiempo es muy intuitivo con esta librería, como veremos a continuación.

In [ ]:
import numpy as np
import pandas as pd

## Fechas

### Dates and times nativos de Python: ``datetime`` and ``dateutil``

Los objetos básicos de Python para trabajar con fechas y horas se encuentran en el módulo ``datetime``.
Junto con el módulo ``dateutil`` se pueden utilizar conjuntamente para una rápida ejecución de un gran número de funcionalidades.
Por ejemplo, puedes construir una fecha manualmente usando el tipo ``datetime``:

In [ ]:
from datetime import datetime
datetime(year=2015, month=7, day=4)

O, usando el módulo ``dateutil``, puedes parsear fechas de una gran variedad de formatos desde un string:

In [ ]:
from dateutil import parser
date = parser.parse("4th of July, 2015")
date

Una vez tienes un objeto ``datetime``, puedes ejecutar acciones como imprimir el día de la semana:

In [ ]:
date.strftime('%A')

En la línea anterior se ha utilizado uno de las codificaciones estándar del formato para imprimir fechas (``"%A"``). Puedes ampliar información sobre este punto en la [sección strftime](https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior) de la documentación de Python sobre [datetime](https://docs.python.org/3/library/datetime.html).
Puedes encontrar documeentación de otras funciones útiles sobre fechas en [la documentación online de dateutil](http://labix.org/python-dateutil).
Un paquete relacionado relevante es [``pytz``](http://pytz.sourceforge.net/), que contiene herramientas para trabajar con uno de los elementos más complejos de las series temporale: las zonas horarias.

El poder de  ``datetime`` y ``dateutil`` viene dado por su flexibilidad y sintaxis fácil: pueds usar estos objetos y sus métodos para llevar a cabo casi cualquier operación. Su limitación viene al trabajar con vectores grandes: las listas de objetos datetime de Python son menos eficiente comparado con arrays tipados de fechas (igual que las listas de números son subóptimas comparadas con las de Numpy). 

### Vectores tipados de fechas en Numpy

El dtype ``datetime64`` codifica fechas como integers de 64-bit, lo que permite que los arrays de fechas sean muy compactos. Este dtyp requiere un formato input muy específico:

In [ ]:
import numpy as np
date = np.array('2015-07-04', dtype=np.datetime64)
date

Sin embargo, una vez formateado es sencillo y rápido realizar operaciones:

In [ ]:
date + np.arange(12)

Debido al tipo uniforme de los vectores de ``datetime64``, este tipo de operacions pueden ser ejecutadas mucho más rápidamente, especialmente para vectores de gran tamaño.

NOTA ADICIONAL: Un detalle de los objetos ``datetime64`` y ``timedelta64`` es que se construyen en una *unidad de tiempo fundamental*. Debido a que el objeto ``datetime64`` se limita a una precisión de 64 bits, el rango de tiempos codificables es $2^{64}$ veces la unidad fundamental. Es decir, este tipo impone un trade-off entre *precisión* de la fecha y *tiempo máximo*.
Por ejemplo, si se quieren guardar fechas con resolución de un nanosegundo, sólo se podrá tener información suficiente para codificar un rango de $2^{64}$ nanosegundos (por debajo de 600 años).

NumPy inferirá la unidad deseada del input. Por ejemplo, una fecha basada en dias:

In [ ]:
np.datetime64('2015-07-04 12:00')

Observa que la zona horaria se establece automáticamente según la zona local del ordenador desde el que se ejecuta.

Puedes forzar el tipo de una fecha a cualquier unidad fundamental deseada. Por ejemplo, a nanosegundos:

In [ ]:
np.datetime64('2015-07-04 12:59:59.50', 'ns')

La siguiente tabla, obtenida de la [doucmentación NumPy datetime64](http://docs.scipy.org/doc/numpy/reference/arrays.datetime.html), lista los formatos disponibles con los límites temporales:

|Code    | Meaning     | Time span (relative) | Time span (absolute)   |
|--------|-------------|----------------------|------------------------|
| ``Y``  | Year	       | ± 9.2e18 years       | [9.2e18 BC, 9.2e18 AD] |
| ``M``  | Month       | ± 7.6e17 years       | [7.6e17 BC, 7.6e17 AD] |
| ``W``  | Week	       | ± 1.7e17 years       | [1.7e17 BC, 1.7e17 AD] |
| ``D``  | Day         | ± 2.5e16 years       | [2.5e16 BC, 2.5e16 AD] |
| ``h``  | Hour        | ± 1.0e15 years       | [1.0e15 BC, 1.0e15 AD] |
| ``m``  | Minute      | ± 1.7e13 years       | [1.7e13 BC, 1.7e13 AD] |
| ``s``  | Second      | ± 2.9e12 years       | [ 2.9e9 BC, 2.9e9 AD]  |
| ``ms`` | Millisecond | ± 2.9e9 years        | [ 2.9e6 BC, 2.9e6 AD]  |
| ``us`` | Microsecond | ± 2.9e6 years        | [290301 BC, 294241 AD] |
| ``ns`` | Nanosecond  | ± 292 years          | [ 1678 AD, 2262 AD]    |
| ``ps`` | Picosecond  | ± 106 days           | [ 1969 AD, 1970 AD]    |
| ``fs`` | Femtosecond | ± 2.6 hours          | [ 1969 AD, 1970 AD]    |
| ``as`` | Attosecond  | ± 9.2 seconds        | [ 1969 AD, 1970 AD]    |

El tipo por defecto más utilizado  es ``datetime64[ns]``, debido a su rango de precisión suficiente.

``datetime64`` también tiene ciertas deficiencias relacionadas con métodos incorporados en ``datetime`` y ``dateutil`` que no se encuentran disponibles en Numpy. Más información en [la documentación de NumPy datetime64](http://docs.scipy.org/doc/numpy/reference/arrays.datetime.html).

### Fechas con Pandas: lo mejor de ambos mundos

Pandas builds upon all the tools just discussed to provide a ``Timestamp`` object, which combines the ease-of-use of ``datetime`` and ``dateutil`` with the efficient storage and vectorized interface of ``numpy.datetime64``.
From a group of these ``Timestamp`` objects, Pandas can construct a ``DatetimeIndex`` that can be used to index data in a ``Series`` or ``DataFrame``; we'll see many examples of this below.

For example, we can use Pandas tools to repeat the demonstration from above.
We can parse a flexibly formatted string date, and use format codes to output the day of the week:

In [ ]:
import pandas as pd
date = pd.to_datetime("4th of July, 2015")
date

In [ ]:
date.strftime('%A')

Additionally, we can do NumPy-style vectorized operations directly on this same object:

In [ ]:
date + pd.to_timedelta(np.arange(12), 'D')

In the next section, we will take a closer look at manipulating time series data with the tools provided by Pandas.

## Pandas para time series

### Indexación 

Las herramientas de series temporales de Pandas se vuelven realmente útiles al utilizar *indexación*.
Por ejemplo, podemos construir un objeto ``Serie`` que tiene un índice temporal:

In [ ]:
index = pd.DatetimeIndex(['2014-07-04', '2014-08-04',
                          '2015-07-04', '2015-08-04'])
data = pd.Series([0, 1, 2, 3], index=index)
data

Ahora que se tiene el tiempo en una ``Serie``, se puede hacer uso de cualquier sistema de indexación, pasando valores que pueden ser convertidos en fechas:

In [ ]:
data['2014-07-04':'2015-07-04']

Hay operaciones de indexación adicionales disponibles sólo para fechas, como pasar un año para obtener los datos de ese año:

In [ ]:
data['2015']

### Estructuras de tiempo 

Las estructuras de datos fundamentales de Pandas para series temporales son:

- ``Timestamp``: reemplazo de ``datetime``, basado en el sistema más eficiente de ``numpy.datetime64``. La estructura del índice asociada será ``DatetimeIndex``.
- ``Period``: para periodos de tiempo. La estructura del índice asociado es ``PeriodIndex``.
- ``Timedelta``: para variación de tiempo o duración (mas eficiente que ``datetime.timedelta``). La estructura del índice asociado es ``TimedeltaIndex``.

Los tipos más básicos son ``Timestamp`` y ``DatetimeIndex``.

La forma más común de crear objetos de este tipo es usando la función ``pd.to_datetime()`` , que puede parsear una gran variedad de formatos. Pasar una fecha a esta función produce un ``Timestamp``; pasar una serie de fechas por defecto devuelve un ``DatetimeIndex``:

In [ ]:
dates = pd.to_datetime([datetime(2015, 7, 3), '4th of July, 2015',
                       '2015-Jul-6', '07-07-2015', '20150708'])
dates

Cualquier ``DatetimeIndex`` puede convertirse en un ``PeriodIndex`` con la función ``to_period()``, añadiendo un código de frecuencia ( ``'D'`` = diario) :

In [ ]:
dates.to_period('D')

Un ``TimedeltaIndex`` se crea, por ejemplo, cuando restamos una fecha de otra:

In [ ]:
dates - dates[0]

### Rangos de fechas: ``pd.date_range()``

Igual que las funciones ``range()`` (nativo) y ``np.arange()`` (NumPy) que generan una secuencia tomando un punto de inicio, un punto de fin y un paso, las funciones ``pd.date_range()`` (para fechas) y ``pd.period_range()`` (para duraciones) funcionan de forma similar.
Así, para crear un vector de fechas, se usa la función 'date_range'. El uso básico es:
- date_range(beginning_date,ending_date), que devolvería una serie temporal diario entre ambas fechas (incluidas),
- date_range(beginning_date, periods=periods) que devolvería un rango de fechas desde beginning_date con tantas fechas como se especifiquen en 'periods'. 

Por defecto, la frecuencia es diaria:

In [ ]:
# Pregunta: el rango incluirá el último día o no?
pd.date_range('2015-07-03', '2015-07-10')

Usandolo a través de la definición del número de periodos:

In [ ]:
pd.date_range('2015-07-03', periods=8)

Además, se puede introducir un atributo ``freq`` para cambiar la frecuencia de las fechas de diaria (``D``) a otras diferentes que se verán en el siguiente apartado. Por ejemplo, a nivel horario: 

In [ ]:
pd.date_range('2015-07-03', periods=8, freq='H')

Para crear secuencias de ``Period`` o ``Timedelta``, se puede usar igualmente ``pd.period_range()`` y ``pd.timedelta_range()``.
Por ejemplo, para periodos mensuales:

In [ ]:
pd.period_range('2015-07', periods=8, freq='M')

Y para secuencias de duración incremental de hora en hora:

In [ ]:
pd.timedelta_range(0, periods=10, freq='H')

### Frecuencias

Ya vimos anteriormente las frecuencias diarias ``D`` y horarias ``H``. La tabla siguiente resume los principales códigos disponibles:

| Code   | Description         | Code   | Description          |
|--------|---------------------|--------|----------------------|
| ``D``  | Calendar day        | ``B``  | Business day         |
| ``W``  | Weekly              |        |                      |
| ``M``  | Month end           | ``BM`` | Business month end   |
| ``Q``  | Quarter end         | ``BQ`` | Business quarter end |
| ``A``  | Year end            | ``BA`` | Business year end    |
| ``H``  | Hours               | ``BH`` | Business hours       |
| ``T``  | Minutes             |        |                      |
| ``S``  | Seconds             |        |                      |
| ``L``  | Milliseonds         |        |                      |
| ``U``  | Microseconds        |        |                      |
| ``N``  | nanoseconds         |        |                      |

Las frecuencias mensual, trimestral y anual se proporcionan como fecha al final del periodo especificado.
Añadiendo una 'S' al final de cada una de estas, la salida será al inicio del periodo.

| Code    | Description            || Code    | Description            |
|---------|------------------------||---------|------------------------|
| ``MS``  | Month start            ||``BMS``  | Business month start   |
| ``QS``  | Quarter start          ||``BQS``  | Business quarter start |
| ``AS``  | Year start             ||``BAS``  | Business year start    |

Además se puede cambiar el mes usado en frecuencias trimestrales o anuales añadiendo un código de tres letras al final:

- ``Q-JAN``, ``BQ-FEB``, ``QS-MAR``, ``BQS-APR``, etc.
- ``A-JAN``, ``BA-FEB``, ``AS-MAR``, ``BAS-APR``, etc.

Igualmente se puede realizar con frecuencias semanales:
    
- ``W-SUN``, ``W-MON``, ``W-TUE``, ``W-WED``, etc.

Y más detallado, se puede combinar con números para otras frecuencias. Por ejemplo para frecuencias de 2 horas y 30 minutos, se puede combinar la hora (``H``) y los minutos (``T``) como sigue:

pd.timedelta_range(0, periods=9, freq="2H30T")

Todos estos códigos pueden encontrarse en el módulo ``pd.tseries.offsets``, y se pueden generar nuevas frecuencias. Por ejemplo, se puede crear un seguimiento de días laborables directamente como se muestra a continuación:

In [ ]:
from pandas.tseries.offsets import BDay
pd.date_range('2015-07-01', periods=5, freq=BDay())

Para más información, [sección "DateOffset"](http://pandas.pydata.org/pandas-docs/stable/timeseries.html#dateoffset-objects) de la documentación de Pandas

### Remuestreo, Desplazamiento y generación de ventanas temporales

La indexación de Pandas permite operaciones de este tipo con relativa facilidad. Para mostrarlo, vamos a generar un vector de fechas con datos ficticios

In [ ]:
dates=pd.date_range('2015-07-25', periods=15, freq='B')
timeser = pd.DataFrame({'close':[10,12,14,15,15,19, 20,17, 15, 14, 12,13,13,14,10]})
timeser.index = dates
timeser

#### Remuestreo

Es común necesitar obtener los datos a mayor o menor frecuencia de la disponible. Para ello se puede usar ``resample()`` o ``asfreq()`` (más sencillo).
La diferencia principal es que mientras la primera es una agregación de datos, la segunda es una selección de los mismos. Sobre nuestros datos anteriores:

In [ ]:
timeser.resample('BM').mean()

In [ ]:
timeser.asfreq('BM')

Mientras que ``resample()`` devuelve la media del periodo, ``asfreq()`` devuelve el último valor (omitiendo el mes en caso de no existir).

Una ventaja adicional de ``asfreq()`` es la capacidad de imputar valores.

#### Desplazamiento

Existen dos métodos principales: ``shift()`` and ``tshift()``
Mientras ``shift()`` desplaza los datos, ``tshift()`` desplaza el índice.
En ambos casos, se especifica por la frecuencia.
Cada 2 días:

In [ ]:
timeser.shift(5)

In [ ]:
timeser.tshift(5)

#### Ventanas móviles

Se utiliza la función ``rolling()``, que funciona de forma similar a ``groupby``, con distintas operaciones de agregación disponibles. Por ejemplo, una media móvil centrada de 3 días:

In [ ]:
timeser.rolling(3, center=True)